In [ ]:
search_company = 'polycab'

In [ ]:
# company search
import requests
import json

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/api/v1/ind/company_search/?q={search_company}", 
    headers=headers
)

search_results = []
slug = ''

if not 200 <= response.status_code < 400:
    print(response.text)
else:
    for item in json.loads(response.text):
        if item['type'] == 'companies':
            search_results.append(
                {
                    'name' : item['name'],
                    'slug' : item['slug']
                }
            )
    if len(search_results) > 1:
        print("Update appropriate slug in next cell")
        print(json.dumps(search_results, ensure_ascii=False, indent=4))
    elif len(search_results) == 1:
        slug = search_results[0]['slug']
        print("Single slug found and loaded, please skip next cell and proceed")
    else:
        print("No matching company found")

In [ ]:
slug = 'polycab-india-ltd'

In [ ]:
# fetch particular company info

from bs4 import BeautifulSoup
import json

headers = headers_dict = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

soup_overview = None
soup_benchmarking = None
soup_financials = None
soup_shareholding = None

response_overview = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}", 
    headers=headers
)

if not 200 <= response_overview.status_code < 400:
    print(response_overview.text)
else:
    print(f"Success fetch overview - response_overview -> https://www.tijorifinance.com/company/{slug}")
    soup_overview = BeautifulSoup(response_overview.text, 'html.parser')

response_financials = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/financials/", 
    headers=headers
)

if not 200 <= response_financials.status_code < 400:
    print(response_financials.text)
else:
    print(f"Success fetch financials - response_financials -> https://www.tijorifinance.com/company/{slug}/financials/")
    soup_financials = BeautifulSoup(response_financials.text, 'html.parser')

response_benchmarking = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/benchmarking/", 
    headers=headers
)

if not 200 <= response_benchmarking.status_code < 400:
    print(response_benchmarking.text)
else:
    print(f"Success fetch benchamarking - response_benchmarking -> https://www.tijorifinance.com/company/{slug}/benchmarking/")
    soup_benchmarking = BeautifulSoup(response_benchmarking.text, 'html.parser')
    
response_shareholding = requests.request(
    "GET", 
    url = f"https://www.tijorifinance.com/company/{slug}/shareholding/", 
    headers=headers
)

if not 200 <= response_shareholding.status_code < 400:
    print(response_shareholding.text)
else:
    print(f"Success fetch shareholding - response_shareholding -> https://www.tijorifinance.com/company/{slug}/shareholding/")
    soup_shareholding = BeautifulSoup(response_shareholding.text, 'html.parser')

In [ ]:
# Get about, market cap, pe ratio

data = {}
data['summary'] = soup_overview.find('div', class_='about_desc detailed_desc').text.strip().split('\n')[0].strip()
data['market_cap'] = (int((soup_overview.find('div', string='Market Cap').find_next('div', class_='company_details_value').text).replace('₹', '').replace(' Cr.', '').replace(',', '')) * 10**7 )
data['pe_ratio'] = float(soup_overview.find('div', string='P/E').find_next('div', class_='company_details_value').text)


print(json.dumps(data, ensure_ascii=False, indent=4))

In [ ]:
# Get all FAQs list
faq_data = []

for soup in [soup_overview, soup_financials, soup_benchmarking, soup_shareholding]:
    
    faq_containers = soup.find_all('div', class_='faq_container')
    
    for container in faq_containers:
        question = container.find('div', class_='faq_data_header').find('span').text.strip()
        answer = container.find('div', class_='faq_answer').find('p').text.strip()
        
        faq_data.append(
            {
                'question': question,
                'answer': answer.split("\n")[0]
            }
        )
    
print(json.dumps(faq_data, ensure_ascii=False, indent=4))

In [ ]:
# Get company forensics ( 1 - good, 2 - neutral, 3 - bad, 4 - not calculated )

script_data = soup_overview.find('script', id='company_details_data', type='application/json').text.strip()
forensics_data = json.loads(script_data)

if forensics_data['quick_look']:
    # Collect the factories
    forensics = forensics_data['quick_look']['data'][0]['factories']
    forensics.extend(forensics_data['quick_look']['data'][1]['factories'])
    
    forensics = sorted(forensics, key=lambda x: x['flag'], reverse=True)
    
    print(json.dumps(forensics, ensure_ascii=False, indent=4))
else:
    print('No forensics data available')

In [ ]:
# Peers data for comparision

import pandas as pd
from IPython.display import HTML

def display_scrollable_dataframe(df_pivot, table_height = 150):
    html = df_pivot.to_html(classes='table table-bordered', index=False)
    display(HTML(f"""
    <div style="height:{table_height}px; overflow:auto; border:1px solid lightgrey; padding:5px;">
        {html}
    </div>
    """))

# soup = BeautifulSoup(response_overview.text, 'html.parser')

script_data = soup_overview.find('script', id='peers_table_data', type='application/json').text.strip()
peers_data = json.loads(script_data)

df_peers_data = pd.DataFrame(peers_data[1]['data'])

display_scrollable_dataframe(df_peers_data)

In [ ]:
# Shareholding data over the years


all_data = []

for script_tag in soup_shareholding.find_all('script'):
    js_code = script_tag.string
    if js_code: 
        try:
            if 'var trendData' in js_code or 'var trendData2' in js_code:
                start_index = js_code.index('[') 
                end_index = js_code.rindex(']') + 1

                json_string = js_code[start_index:end_index]
                data = json.loads(json_string.replace("'", '"')) 

                all_data.append(data)
        except Exception as e:
            print(f"Error extracting JSON from script: {e}")

flattened_data = []
for dataset in all_data:
    for entry in dataset:
        name = entry['name']
        for item in entry['data']:
            timestamp = item[0]  # The timestamp
            value = item[1]      # The associated value
            flattened_data.append({'Name': name, 'Timestamp': timestamp, 'Value': value})



df = pd.DataFrame(flattened_data)
df['DateIST'] = pd.to_datetime(df['Timestamp'], unit='ms') + pd.Timedelta(hours=5, minutes=30)
df['DateIST'] = df['DateIST'].dt.strftime('%Y-%m')

df_shareholder_info = df.pivot_table(index='Name', columns='DateIST', values='Value')
df_shareholder_info.reset_index(inplace=True)

display_scrollable_dataframe(df_shareholder_info)

In [ ]:
# Load Financial data

script_data = soup_financials.find('script', id='fin_tables_data', type='application/json').text.strip()
fin_data = json.loads(script_data)

def process_section(report_dates,entry_list,name = 'Overall'): # pass list

    #report_dates.sort(reverse = True)

    values_dict = {date: [] for date in report_dates}  # Dictionary to hold values for each report date

    df = pd.DataFrame()
    
    # Iterate through each subsection
    for section in entry_list:
        sub_section_name = section['name']  # Category name
        
        # Add the values for each report date
        for i, date in enumerate(report_dates):
            if i < len(section['value']) and section['value'][i]:  
                values_dict[date].append(round(section['value'][i],2))
            else:
                values_dict[date].append(0)

        df = pd.concat(
                [df,pd.DataFrame({
                    'category': sub_section_name,
                    **values_dict 
                })],
                ignore_index = True
            )

        values_dict = {date: [] for date in report_dates}

        if section['sub_section']:
            df = pd.concat(
                    [df,process_section(report_dates,section['sub_section'], sub_section_name)],
                    ignore_index = True
                )

    return df

def get_complete_report(report_key = 'pl_c_d'):
    dict_shorts_0 = {
        'pl' : 'Profit and Loss',
        'bs' : 'Balance Sheet',
        'cf' : 'Cash Flow',
        'qt' : 'Quarterly',
        'growth' : 'Growth Table',
        'fr' : 'Financial Ratio'
    }
    
    dict_shorts_1 = {
        'c' : 'consolidated',
        's' : 'standalone'
    }
    
    dict_shorts_2 = {
        's' : 'summary',
        'd' : 'detail'
    }
    
    report_name = ''
    if len(report_key.split("_")) > 0:
        report_name += dict_shorts_0[report_key.split("_")[0]]+' '
    if len(report_key.split("_")) > 1:
        if report_key.split("_")[1] == 's':
            print(f'Skipping standalone report for {report_name}')
            return
            
        report_name += dict_shorts_1[report_key.split("_")[1]]+' '
    if len(report_key.split("_")) > 2:
        report_name += dict_shorts_2[report_key.split("_")[2]]+' '
        if report_key.split("_")[2] == 's':
            print(f'Skipping summary report for {report_name}')
            return
            
    report_name += 'statement'

    print("\n")
    print(report_name)
    if fin_data[report_key]:
        display_scrollable_dataframe(
            process_section(
                fin_data[report_key]['report_dates'],
                fin_data[report_key]['data'],
                name
            ),
            400
        )
    else:
        print("No data available")

for key in fin_data.keys():
    get_complete_report(key)